# **Text Classification**
## **BIG DATA PYSPARK** 2021
### **Final Project**
### **Artur Avagyan**

In [0]:
from pyspark import SparkContext,SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [0]:
df = spark.sql("select * from tert_data_csv")

In [0]:
df.printSchema()

root
-- Website: string (nullable = true)
-- Category: string (nullable = true)
-- Year: integer (nullable = true)
-- Month: string (nullable = true)
-- Weekday: string (nullable = true)
-- Day: integer (nullable = true)
-- Hour: integer (nullable = true)
-- Title: string (nullable = true)
-- Article: string (nullable = true)

In [0]:
print('Count of rows: ',df.count(),'\nCount of columns: ', len(df.columns))

Count of rows: 15000 
Count of columns: 9

In [0]:
df.show(10)

+-------+----------------+----+-----+-------+---+----+--------------------+--------------------+
Website| Category|Year|Month|Weekday|Day|Hour| Title| Article|
+-------+----------------+----+-----+-------+---+----+--------------------+--------------------+
tert.am|Քաղաքականություն|2021| June| Sunday| 6| 23|Հիմա ադրբեջանցինե...|Այո, այս ստահակնե...|
tert.am| Իրադարձային|2021| June| Sunday| 6| 23|Էջմիածինում «Rena...|Երեկ՝ հունիսի 5-ի...|
tert.am|Քաղաքականություն|2021| June| Sunday| 6| 23|Ադրբեջանը քաղաքակ...|Ադրբեջանը շարունա...|
tert.am|Քաղաքականություն|2021| June| Sunday| 6| 22|Ես երդվում եմ՝ եթ...|Ես երդվում եմ՝ եթ...|
tert.am|Քաղաքականություն|2021| June| Sunday| 6| 22|Պառակտված ժողովու...|Մենք, երևի թե, մե...|
tert.am| Ժամանց|2021| June| Sunday| 6| 22|Ծնվել է արքայազն ...|Մեծ Բրիտանիայի ար...|
tert.am|Քաղաքականություն|2021| June| Sunday| 6| 21|Բարև, ՀՀ քաղաքացի...|Նիկոլ Փաշինյանը ֆ...|
tert.am| Իրավունք|2021| June| Sunday| 6| 21|Ադրբեջանական զինվ...|ՀՀ մարդու իրավուն...|
tert.am| Իրավունք|2021| June| Sunday| 6| 21|Ադրբեջանական զինվ...|ՀՀ մարդու իրավուն...|
tert.am|Քաղաքականություն|2021| June| Sunday| 6| 21|«Պատիվ ունեմ» դաշ...|«ՊԱՏԻՎ ՈՒՆԵՄ» դաշ...|
+-------+----------------+----+-----+-------+---+----+--------------------+--------------------+
only showing top 10 rows

In [0]:
df.select("Category") \
    .distinct().show()

+------------------+
 Category|
+------------------+
 Սպորտ |
 Իրադարձային|
 Կորոնավիրուս|
 Քաղաքականություն|
Մամուլի տեսություն|
 Քաղաքականություն|
Գիտություն/տեխնիկա|
 Սպորտ|
 Իրադարձային |
 Քաղաքականություն |
 Իրավունք|
 Մշակույթ|
 Ժամանց|
 Սպորտ|
 Իրադարձային|
 Տնտեսություն|
+------------------+

In [0]:
df = df.withColumn("Category", when(df.Category == ' Քաղաքականություն','Քաղաքականություն')\
                               .when(df.Category == 'Քաղաքականություն ','Քաղաքականություն')\
                               .when(df.Category == ' Իրադարձային','Իրադարձային')\
                               .when(df.Category == 'Իրադարձային ','Իրադարձային')\
                               .when(df.Category == ' Սպորտ','Սպորտ')\
                               .when(df.Category == 'Սպորտ ','Սպորտ')\
                               .otherwise(df.Category))
df=df.withColumn('Month', when(df.Month=='Febuary','February').otherwise(df.Month))

In [0]:
df.select("Category") \
    .distinct().show()

+------------------+
 Category|
+------------------+
 Իրադարձային|
 Կորոնավիրուս|
 Քաղաքականություն|
Մամուլի տեսություն|
Գիտություն/տեխնիկա|
 Իրավունք|
 Մշակույթ|
 Ժամանց|
 Սպորտ|
 Տնտեսություն|
+------------------+

In [0]:
df.groupBy('Category').count().show()

+------------------+-----+
 Category|count|
+------------------+-----+
 Իրադարձային| 7035|
 Կորոնավիրուս| 131|
 Քաղաքականություն| 2710|
Մամուլի տեսություն| 868|
Գիտություն/տեխնիկա| 407|
 Իրավունք| 593|
 Մշակույթ| 185|
 Ժամանց| 863|
 Սպորտ| 1508|
 Տնտեսություն| 700|
+------------------+-----+

In [0]:
for col in df.columns:
  print(col, '  with null values: ', df.filter(df[col].isNull()).count())

Website with null values: 0
Category with null values: 0
Year with null values: 0
Month with null values: 0
Weekday with null values: 0
Day with null values: 0
Hour with null values: 0
Title with null values: 0
Article with null values: 0

In [0]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
tert_data=df.toPandas()
fig=make_subplots(rows=2,cols=2,
                subplot_titles=("Article distribution by categories",
                                "Article distribution by month",
                                "Article distribution by weekday",
                                "Article distribution by hour"))
fig.add_trace(go.Bar(y=tert_data.Category.value_counts().sort_values(ascending=True).index,
                    x=tert_data.Category.value_counts().sort_values(ascending=True),
                    name='',
                    orientation='h',
                    showlegend=False,
                    text=tert_data.Category.value_counts().sort_values(ascending=True),
                    textposition='outside'),row=1,col=1)

tert_data['Month'] = pd.Categorical(tert_data['Month'],
                            categories=['January','February','March','April','May','June',
                                        'July','August','September','October','November','December'],ordered=True)
fig.add_trace(go.Bar(x=tert_data.Month.value_counts().sort_index(ascending=True).index,
                    y=tert_data.Month.value_counts().sort_index(ascending=True),
                    name='',
                    orientation='v',
                    showlegend=False,
                    text=tert_data.Month.value_counts().sort_index(ascending=True),
                    textposition='outside'),row=1,col=2)

tert_data['Weekday'] = pd.Categorical(tert_data['Weekday'],
                            categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],ordered=True)
fig.add_trace(go.Bar(x=tert_data.Weekday.value_counts().sort_index(ascending=True).index,
                    y=tert_data.Weekday.value_counts().sort_index(ascending=True),
                    name='',
                    orientation='v',
                    showlegend=False,
                    text=tert_data.Weekday.value_counts().sort_index(ascending=True),
                    textposition='outside'),row=2,col=1)

tert_data['Hour'] = pd.Categorical(tert_data['Hour'],
                            categories=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],ordered=True)
fig.add_trace(go.Bar(x=tert_data.Hour.value_counts().sort_index(ascending=True).index,
                    y=tert_data.Hour.value_counts().sort_index(ascending=True),
                    name='',
                    orientation='v',
                    showlegend=False,
                    text=tert_data.Hour.value_counts().sort_index(ascending=True),
                    textposition='outside'),row=2,col=2)


fig.update_layout(title="Some statistics from tert.am",
                  width=1650, height=950,
                  # paper_bgcolor="LightSteelBlue"
                  )
fig.show()

In [0]:
# df=df.withColumn("Year",df.Year.cast('integer'))\
#   .withColumn("Day",df.Day.cast('integer'))\
#   .withColumn("Hour",df.Hour.cast('integer'))

In [0]:
df_only_text=df.select('Article','Category')
df_only_text.show(15)

+--------------------+----------------+
 Article| Category|
+--------------------+----------------+
Այո, այս ստահակնե...|Քաղաքականություն|
Երեկ՝ հունիսի 5-ի...| Իրադարձային|
Ադրբեջանը շարունա...|Քաղաքականություն|
Ես երդվում եմ՝ եթ...|Քաղաքականություն|
Մենք, երևի թե, մե...|Քաղաքականություն|
Մեծ Բրիտանիայի ար...| Ժամանց|
Նիկոլ Փաշինյանը ֆ...|Քաղաքականություն|
ՀՀ մարդու իրավուն...| Իրավունք|
ՀՀ մարդու իրավուն...| Իրավունք|
«ՊԱՏԻՎ ՈՒՆԵՄ» դաշ...|Քաղաքականություն|
Ամերիկյան P-8A Po...| Իրադարձային|
Ռուս-ամերիկյան գա...| Իրադարձային|
Արցախյան 44-օրյա ...| Իրադարձային|
Թենիսի Մեծ Սաղավա...| Սպորտ|
Իմ քաղաքական հայա...| Իրադարձային|
+--------------------+----------------+
only showing top 15 rows

In [0]:
df_only_text = df_only_text.withColumn("Category", when(df.Category == 'Քաղաքականություն','Քաղաքական')\
                               .when(df.Category == 'Իրադարձային','Քաղաքական')\
                               .otherwise('Ոչ քաղաքական'))

## Data preprocessing and ML models

In [0]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover,CountVectorizer,IDF, StringIndexer

tokenizer=Tokenizer(inputCol='Article',outputCol='mytokens')

arm_stop_words=["այդ","այլ","այն","այս","դու","դուք","եմ","չեմ","երևի",
            "են","ենք","ես","եք","է","էի","էին","էինք","եթե","դա",
            "չեն","չենք","չես","չեք","չի","չէի","չէին","չէինք",
            "էիր","էիք","էր","ըստ","թ.","թե","ի","ին","իսկ","ինչ","դրա",
            "իր","կամ","համար","հետ","հետո","մենք","մեջ","մասին","մեր",
            "մի","նա","նաև","նրա","նրանք","որ","որը","չի","դեպքում","ու",
            "որոնք","որպես","ում","պիտի","վրա","և","եւ","կարող","սակայն",
            "ահա", "անգամ", "գեթ", "գոնե", "իբր", "թող", "լոկ","որովհետև",
            "խո", "խոմ", "հենց", "հո", "մանավանդ","իրոք որ", "թերևս", "թեկուզ",
            "կարծես", "միգուցե", "միթե", "մինչև","անգամ", "նույնիսկ",
            "անկասկած", "անպատճառ", "անտարակույս","իսկապես", "միայն"]

stopwords_remover=StopWordsRemover(inputCol='mytokens',outputCol='filtered_tokens',stopWords=arm_stop_words)
vectorizer=CountVectorizer(inputCol='filtered_tokens',outputCol='rawFeatures')
idf=IDF(inputCol='rawFeatures',outputCol='vectorizedFeatures')


labelEncoder = StringIndexer(inputCol='Category',outputCol='label').fit(df_only_text)
labelEncoder.labels

Out[70]: ['Քաղաքական', 'Ոչ քաղաքական']

In [0]:
df_only_text=labelEncoder.transform(df_only_text)
df_only_text.show(5)

+--------------------+---------+-----+
 Article| Category|label|
+--------------------+---------+-----+
Այո, այս ստահակնե...|Քաղաքական| 0.0|
Երեկ՝ հունիսի 5-ի...|Քաղաքական| 0.0|
Ադրբեջանը շարունա...|Քաղաքական| 0.0|
Ես երդվում եմ՝ եթ...|Քաղաքական| 0.0|
Մենք, երևի թե, մե...|Քաղաքական| 0.0|
+--------------------+---------+-----+
only showing top 5 rows

In [0]:
(trainDF, testDF)=df_only_text.randomSplit((0.75,0.25),seed=42)
trainDF.show(5)

+-------+------------+-----+
Article| Category|label|
+-------+------------+-----+
 |Ոչ քաղաքական| 1.0|
 | Քաղաքական| 0.0|
 | Քաղաքական| 0.0|
 | Քաղաքական| 0.0|
 | Քաղաքական| 0.0|
+-------+------------+-----+
only showing top 5 rows

## Text classification models

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
lr=LogisticRegression(featuresCol='vectorizedFeatures',labelCol='label')
pipeline=Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,lr])
lr_model=pipeline.fit(trainDF)
predictions=lr_model.transform(testDF)
predictions.columns

Out[76]: ['Article',
 'Category',
 'label',
 'mytokens',
 'filtered_tokens',
 'rawFeatures',
 'vectorizedFeatures',
 'rawPrediction',
 'probability',
 'prediction']

In [0]:
predictions.select( 'probability', 'Category','label','prediction').show(10)

+--------------------+---------+-----+----------+
 probability| Category|label|prediction|
+--------------------+---------+-----+----------+
[0.95833184954488...|Քաղաքական| 0.0| 0.0|
[0.95833184954488...|Քաղաքական| 0.0| 0.0|
[0.95833184954488...|Քաղաքական| 0.0| 0.0|
[0.95833184954488...|Քաղաքական| 0.0| 0.0|
[0.95833184954488...|Քաղաքական| 0.0| 0.0|
[0.95833184954488...|Քաղաքական| 0.0| 0.0|
[0.95833184954488...|Քաղաքական| 0.0| 0.0|
[0.95833184954488...|Քաղաքական| 0.0| 0.0|
[0.95833184954488...|Քաղաքական| 0.0| 0.0|
[0.95833184954488...|Քաղաքական| 0.0| 0.0|
+--------------------+---------+-----+----------+
only showing top 10 rows

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator=MulticlassClassificationEvaluator(labelCol='label',
                                           predictionCol='prediction',
                                           metricName='accuracy')
accuracy=evaluator.evaluate(predictions)
accuracy

Out[79]: 0.8644390111382776

In [0]:
from pyspark.mllib.evaluation import MulticlassMetrics
lr_metric=MulticlassMetrics(predictions['label','prediction'].rdd)
print('Accuracy: --> ',lr_metric.accuracy)
print('Precision for label 0: --> ',lr_metric.precision(0.0))
print('Precision for label 1: --> ',lr_metric.precision(1.0))
print('Recall for label 0: --> ',lr_metric.recall(0.0))
print('Recall for label 1: --> ',lr_metric.recall(1.0))
print('F1Score for label 0: --> ',lr_metric.fMeasure(0.0))
print('F1Score for label 1: --> ',lr_metric.fMeasure(1.0))
print('Confusion Matrix: -->\n',lr_metric.confusionMatrix().toArray())

from sklearn.metrics import classification_report
print('\nClassification Report\n',classification_report(predictions.select('label').toPandas(),
                            predictions.select('prediction').toPandas(),
                            target_names=labelEncoder.labels))

Accuracy: --> 0.8644390111382776
Precision for label 0: --> 0.9581433899709905
Precision for label 1: --> 0.6861198738170347
Recall for label 0: --> 0.8531365313653136
Recall for label 1: --> 0.8959835221421215
F1Score for label 0: --> 0.9025961350771032
F1Score for label 1: --> 0.7771326485037963
Confusion Matrix: -->
 [[2312. 398.]
 [ 101. 870.]]

Classification Report
 precision recall f1-score support

 Քաղաքական 0.85 0.96 0.90 2413
Ոչ քաղաքական 0.90 0.69 0.78 1268

 accuracy 0.86 3681
 macro avg 0.87 0.82 0.84 3681
weighted avg 0.87 0.86 0.86 3681

In [0]:
from pyspark.sql.types import StringType
example_article=spark.createDataFrame([
  ('''
  Ռամոսի հետ պայմանագիր կնքելու ցանկություն ունեն անգլիական «Մանչեսթեր Սիթին» ու «Մանչեսթեր Յունայթեդը» եւ ֆրանսիական ՊՍԺ-ն, հաղորդել էր իսպանական AS-ը։
  Սերխիո Ռամոսը 2005 թվականին «Սեւիլիայից» է տեղափոխվել «Ռեալ», որի կազմում անցկացրել է 671 հանդիպում, խփել 101 գոլ, կատարել 40 գոլային փոխանցում, դարձել Իսպանիայի հնգակի չեմպիոն, կրկնակի գավաթակիր, քառակի սուպերգավաթակիր, Չեմպիոնների լիգայի քառակի գավաթակիր, Եվրոպայի եռակի սուպերգավաթակիր, աշխարհի քառակի ակումբային չեմպիոն։
  ''',StringType())
],['Article'])

pred_example=lr_model.transform(example_article)
pred_example.select('Article','prediction')\
  .withColumn("prediction", when(pred_example.prediction == 0.0,'Քաղաքական')\
                               .otherwise('Ոչ քաղաքական')).show()


+--------------------+------------+
 Article| prediction|
+--------------------+------------+

 Ռամոսի հետ պայ...|Ոչ քաղաքական|
+--------------------+------------+

### Other Classification models

In [0]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(featuresCol='vectorizedFeatures',labelCol='label',maxDepth=10)
pipeline=Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,dt])
dt_model=pipeline.fit(trainDF)
predictions=dt_model.transform(testDF)
evaluator=MulticlassClassificationEvaluator(labelCol='label',
                                           predictionCol='prediction',
                                           metricName='accuracy')
accuracy=evaluator.evaluate(predictions)
accuracy

Out[82]: 0.8429774517794077

In [0]:
dt_metric=MulticlassMetrics(predictions['label','prediction'].rdd)
print('Accuracy: --> ',dt_metric.accuracy)
print('Precision for label 0: --> ',dt_metric.precision(0.0))
print('Precision for label 1: --> ',dt_metric.precision(1.0))
print('Recall for label 0: --> ',dt_metric.recall(0.0))
print('Recall for label 1: --> ',dt_metric.recall(1.0))
print('F1Score for label 0: --> ',dt_metric.fMeasure(0.0))
print('F1Score for label 1: --> ',dt_metric.fMeasure(1.0))
print('Confusion Matrix: -->\n',dt_metric.confusionMatrix().toArray())

print('\nClassification Report\n',classification_report(predictions.select('label').toPandas(),
                            predictions.select('prediction').toPandas(),
                            target_names=labelEncoder.labels))

Accuracy: --> 0.8429774517794077
Precision for label 0: --> 0.9697472026523001
Precision for label 1: --> 0.6017350157728707
Recall for label 0: --> 0.8224956063268892
Recall for label 1: --> 0.9126794258373205
F1Score for label 0: --> 0.8900722708254089
F1Score for label 1: --> 0.7252851711026616
Confusion Matrix: -->
 [[2340. 505.]
 [ 73. 763.]]

Classification Report
 precision recall f1-score support

 Քաղաքական 0.82 0.97 0.89 2413
Ոչ քաղաքական 0.91 0.60 0.73 1268

 accuracy 0.84 3681
 macro avg 0.87 0.79 0.81 3681
weighted avg 0.85 0.84 0.83 3681

In [0]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol='vectorizedFeatures',labelCol='label', numTrees=100,maxDepth=10)
pipeline=Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,rf])
rf_model=pipeline.fit(trainDF)
predictions=rf_model.transform(testDF)
evaluator=MulticlassClassificationEvaluator(labelCol='label',
                                           predictionCol='prediction',
                                           metricName='accuracy')
accuracy=evaluator.evaluate(predictions)
accuracy

Out[87]: 0.6883998913338767

In [0]:
rf_metric=MulticlassMetrics(predictions['label','prediction'].rdd)
print('Accuracy: --> ',rf_metric.accuracy)
print('Precision for label 0: --> ',rf_metric.precision(0.0))
print('Precision for label 1: --> ',rf_metric.precision(1.0))
print('Recall for label 0: --> ',rf_metric.recall(0.0))
print('Recall for label 1: --> ',rf_metric.recall(1.0))
print('F1Score for label 0: --> ',rf_metric.fMeasure(0.0))
print('F1Score for label 1: --> ',rf_metric.fMeasure(1.0))
print('Confusion Matrix: -->\n',rf_metric.confusionMatrix().toArray())

print('\nClassification Report\n',classification_report(predictions.select('label').toPandas(),
                            predictions.select('prediction').toPandas(),
                            target_names=labelEncoder.labels))

Accuracy: --> 0.6883998913338767
Precision for label 0: --> 1.0
Precision for label 1: --> 0.09542586750788644
Recall for label 0: --> 0.6778089887640449
Recall for label 1: --> 1.0
F1Score for label 0: --> 0.8079691947095261
F1Score for label 1: --> 0.1742260619150468
Confusion Matrix: -->
 [[2413. 1147.]
 [ 0. 121.]]

Classification Report
 precision recall f1-score support

 Քաղաքական 0.68 1.00 0.81 2413
Ոչ քաղաքական 1.00 0.10 0.17 1268

 accuracy 0.69 3681
 macro avg 0.84 0.55 0.49 3681
weighted avg 0.79 0.69 0.59 3681

In [0]:
from pyspark.ml.classification import NaiveBayes
scores=[]
for i in np.arange(0.1, 1.1, 0.1):
  nb = NaiveBayes(featuresCol='vectorizedFeatures',labelCol='label',smoothing=i, modelType="multinomial")
  pipeline=Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,nb])
  nb_model=pipeline.fit(trainDF)
  predictions=nb_model.transform(testDF)
  evaluator=MulticlassClassificationEvaluator(labelCol='label',
                                             predictionCol='prediction',
                                             metricName='accuracy')
  accuracy=evaluator.evaluate(predictions)
  scores.append(accuracy)
alphas = np.arange(0.1,1.1,0.1)
fig = px.line(x=alphas, y=scores,
              )
fig.update_layout(title='Accuracy with different alphas',
                  width=500, height=500,
                  xaxis_title="Alpha",
                  yaxis_title="Accuracy"
                  )
fig.show()

In [0]:
nb = NaiveBayes(featuresCol='vectorizedFeatures',labelCol='label',smoothing=0.9, modelType="multinomial")
pipeline=Pipeline(stages=[tokenizer,stopwords_remover,vectorizer,idf,nb])
nb_model=pipeline.fit(trainDF)
predictions=nb_model.transform(testDF)

nb_metric=MulticlassMetrics(predictions['label','prediction'].rdd)
print('Accuracy: --> ',nb_metric.accuracy)
print('Precision for label 0: --> ',nb_metric.precision(0.0))
print('Precision for label 1: --> ',nb_metric.precision(1.0))
print('Recall for label 0: --> ',nb_metric.recall(0.0))
print('Recall for label 1: --> ',nb_metric.recall(1.0))
print('F1Score for label 0: --> ',nb_metric.fMeasure(0.0))
print('F1Score for label 1: --> ',nb_metric.fMeasure(1.0))
print('Confusion Matrix: -->\n',nb_metric.confusionMatrix().toArray())

print('\nClassification Report\n',classification_report(predictions.select('label').toPandas(),
                            predictions.select('prediction').toPandas(),
                            target_names=labelEncoder.labels))

Accuracy: --> 0.865254007063298
Precision for label 0: --> 0.9411520928305015
Precision for label 1: --> 0.7208201892744479
Recall for label 0: --> 0.8651428571428571
Recall for label 1: --> 0.865530303030303
F1Score for label 0: --> 0.9015482334259628
F1Score for label 1: --> 0.7865748709122202
Confusion Matrix: -->
 [[2271. 354.]
 [ 142. 914.]]

Classification Report
 precision recall f1-score support

 Քաղաքական 0.87 0.94 0.90 2413
Ոչ քաղաքական 0.87 0.72 0.79 1268

 accuracy 0.87 3681
 macro avg 0.87 0.83 0.84 3681
weighted avg 0.87 0.87 0.86 3681

## Best Regards

<h4 align="left">Artur Avagyan</h4>
    <ul>
    <li>Actuarial supervisor at Central Bank of Armenia</li>
    <li>Student from YSU (Data Science for Business Master's Degree Program)</li>
    <li>E-mail:   avagyan.artur97@gmail.com</li>
    <li>LinkedIn: <a href="https://www.linkedin.com/in/artur-avagyan-0a16311b3">Artur Avagyan</a></li>
    <li>GitHub:   <a href="https://github.com/artur-avagyan">Artur Avagyan</a></li>
    <li>Phone:    +37498641815</li>
    </ul>

<h3 align="center">YSU DSB 2020-2021</h3>